### 3.1 Importing all the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score ,f1_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from __future__ import print_function
from sklearn.neural_network import MLPClassifier
from sklearn import datasets


: 

## 4.0 Now, let us load the required data into individual variables

In [21]:
X_train=pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_train_X.csv")
X_test = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_test_X.csv")
y_train = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_train_y.csv")
y_test = pd.read_csv("C:/Users/surya/OneDrive/Documents/USF/Classes/Semester-2/DSP/Assignment-1/data/online_price_test_y.csv")

Let us create a data frame called performance and put all the needed metrics in it!

In [22]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

# 4.1.1 Logistic Regression with Random Search

In [23]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'penalty': ['None','l1','l2','elasticnet'],
    'solver':['saga','liblinear']
}

model = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=500. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits
The best recall score is 0.8657534246575344
... with parameters: {'solver': 'liblinear', 'penalty': 'l1'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

# 4.1.2 Logistic Regression with Grid Search

In [24]:
score_measure = "recall"
kfolds = 5
penalty = rand_search.best_params_['penalty']
solver = rand_search.best_params_['solver']

param_grid = {  
    'penalty': [penalty],
    'solver': [solver]
}

model = LogisticRegression()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallLogistic = grid_search.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
The best recall score is 0.8657534246575344
... with parameters: {'penalty': 'l1', 'solver': 'liblinear'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [25]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression",
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.905537,0.902208,0.913738,0.907937


## 4.2.1 SVM (Linear) - Random Search

In [26]:
score_measure = "recall"
kfolds = 2

param_grid = {
    'C': np.arange(1,20),   
    'gamma': ['scale','auto'],
    'kernel':['linear']
}

model = SVC()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=50,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 2 folds for each of 38 candidates, totalling 76 fits


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 38 is smaller than n_iter=50. Running 38 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best recall score is 0.8684931506849315
... with parameters: {'kernel': 'linear', 'gamma': 'scale', 'C': 3}


## 4.2.2 SVM (Linear) - Grid Search

In [27]:
score_measure = "recall"
kfolds = 5

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-3,C+3),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

model = SVC()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallSVM_linear = grid_search.best_estimator_

Fitting 5 folds for each of 6 candidates, totalling 30 fits
The best recall score is 0.8671232876712329
... with parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packag

In [28]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM-Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.905537,0.902208,0.913738,0.907937
0,SVM-Linear,0.920195,0.945946,0.894569,0.919540


## 4.3.1 SVM (RBF) - Random Search

In [29]:
score_measure = "recall"
kfolds = 2

param_grid = {
    'C': np.arange(1,20),   
    'gamma': ['scale','auto'],
    'kernel':['rbf']
}

model = SVC()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=50,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 2 folds for each of 38 candidates, totalling 76 fits


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 38 is smaller than n_iter=50. Running 38 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


The best recall score is 0.8931506849315068
... with parameters: {'kernel': 'rbf', 'gamma': 'auto', 'C': 18}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 4.3.2 SVM (RBF) - Grid Search

In [30]:
score_measure = "recall"
kfolds = 5

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-3,C+3),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

model = SVC()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallSVM_rbf = grid_search.best_estimator_

Fitting 5 folds for each of 6 candidates, totalling 30 fits
The best recall score is 0.9095890410958904
... with parameters: {'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM-RBF", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.905537,0.902208,0.913738,0.907937
0,SVM-Linear,0.920195,0.945946,0.894569,0.919540
0,SVM-RBF,0.903909,0.891975,0.923323,0.907378


## 4.4.1 SVM (Poly) - Random Search

In [32]:
score_measure = "recall"
kfolds = 2

param_grid = {
    'gamma':[1e-2, 1e-3, 1e-4, 1e-5],
    'C':[0.001, 0.10, 0.0001,0.00001],
    'kernel':['poly']
}

model = SVC()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=10,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 2 folds for each of 10 candidates, totalling 20 fits


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best recall score is 0.8671232876712329
... with parameters: {'kernel': 'poly', 'gamma': 0.001, 'C': 0.1}


## 4.4.2 SVM (Poly) - Grid Search

In [33]:
score_measure = "recall"
kfolds = 5

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-3,C+3),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

model = SVC()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallSVM_poly = grid_search.best_estimator_

Fitting 5 folds for each of 6 candidates, totalling 30 fits
The best recall score is 0.8712328767123287
... with parameters: {'C': 0.10000000000000009, 'gamma': 0.001, 'kernel': 'poly'}


C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\surya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

In [34]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM-Poly", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.905537,0.902208,0.913738,0.907937
0,SVM-Linear,0.920195,0.945946,0.894569,0.919540
0,SVM-RBF,0.903909,0.891975,0.923323,0.907378
0,SVM-Poly,0.915309,0.930693,0.900958,0.915584


## 4.5.1 Decision Trees - Random Search 

In [35]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,100),  
    'min_samples_leaf': np.arange(1,75),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 80), 
    'max_depth': np.arange(1,40), 
    'criterion': ['entropy', 'gini'],
}

model = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = model, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.9178082191780822
... with parameters: {'min_samples_split': 7, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 29, 'max_depth': 18, 'criterion': 'gini'}


## 4.5.2 DECISION TREE - GRID SEARCH

In [36]:
score_measure = "recall"
kfolds = 5
min_samples_split = rand_search.best_params_['min_samples_split']
min_samples_leaf = rand_search.best_params_['min_samples_leaf']
min_impurity_decrease = rand_search.best_params_['min_impurity_decrease']
max_leaf_nodes = rand_search.best_params_['max_leaf_nodes']
max_depth = rand_search.best_params_['max_depth']
criterion = rand_search.best_params_['criterion']

param_grid = {
    'min_samples_split': np.arange(min_samples_split-2,min_samples_split+2),  
    'min_samples_leaf': np.arange(min_samples_leaf-2,min_samples_leaf+2),
    'min_impurity_decrease': np.arange(min_impurity_decrease-0.0001, min_impurity_decrease+0.0001, 0.00005),
    'max_leaf_nodes': np.arange(max_leaf_nodes-2,max_leaf_nodes+2), 
    'max_depth': np.arange(max_depth-2,max_depth+2), 
    'criterion': [criterion]
}

model = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallDTree = grid_search.best_estimator_

Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
The best recall score is 0.9219178082191781
... with parameters: {'criterion': 'gini', 'max_depth': 16, 'max_leaf_nodes': 29, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 3, 'min_samples_split': 8}


In [37]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [38]:
performance

,model,Accuracy,Precision,Recall,F1
0,Logistic Regression,0.905537,0.902208,0.913738,0.907937
0,SVM-Linear,0.920195,0.945946,0.894569,0.919540
0,SVM-RBF,0.903909,0.891975,0.923323,0.907378
0,SVM-Poly,0.915309,0.930693,0.900958,0.915584
0,Decision Tree,0.944625,0.937304,0.955272,0.946203


# 5.0 ANALYSIS

Here, we can see that the Decision Trees have the best recall score of all the models that we have tried. We achieved a sore of 95.52 using decision trees.

The Next best metric we have is SupportVectorMachine using RBF Kernel. But let us go with Decision Trees as we have better score there in addition to the fact that Decision Trees deal with colinear data in a better way

Our Final goal is to achieve the best recall score so that we can reduce False Negatives as much as we can. Theoretically, we can achieve a recall score upto 1 and in reality, we achieved it upto 0.955. So, we have dealt with the issue in the best possible way we can and reduced the occurances of false negatives.